In [59]:
from nnsight import LanguageModel
import circuitsvis

In [60]:
import os
from nnsight import CONFIG

CONFIG.API.APIKEY = input("Enter your API key: ")

In [70]:
model = LanguageModel(
    'meta-llama/Llama-3.2-3B',
    attn_implementation="eager"
)
model.to_empty(device='mps')
model.device

device(type='mps', index=0)

In [62]:
for string in ["frm", "qi", "sx", "ora", "re", "op"]:
    print(model.tokenizer(string, add_special_tokens=False))

{'input_ids': [32736], 'attention_mask': [1]}
{'input_ids': [41287], 'attention_mask': [1]}
{'input_ids': [49712], 'attention_mask': [1]}
{'input_ids': [6347], 'attention_mask': [1]}
{'input_ids': [265], 'attention_mask': [1]}
{'input_ids': [454], 'attention_mask': [1]}


In [63]:
prompt = "If the frm is qi, then sx is ora. If the frm is re, then sx is op. The frm is qi. Therefore sx is"

with model.trace(prompt):
    out = model.output[0].save()
print(out)

Loading checkpoint shards: 100%|██████████| 2/2 [00:40<00:00, 20.38s/it]


tensor([[[ 4.7628,  6.8193, 11.1468,  ..., -4.9572, -4.9566, -4.9570],
         [ 5.6407,  3.7500,  2.0542,  ..., -3.2369, -3.2369, -3.2368],
         [ 2.5235,  3.5541,  1.0761,  ..., -3.6749, -3.6748, -3.6749],
         ...,
         [ 7.7994,  7.2150,  5.3846,  ..., -3.8279, -3.8273, -3.8272],
         [ 8.5459,  8.6300,  8.4415,  ..., -4.2414, -4.2404, -4.2409],
         [ 6.9775,  6.8855,  4.4829,  ..., -4.3135, -4.3134, -4.3135]]],
       grad_fn=<UnsafeViewBackward0>)


In [64]:
import torch

# Access the logits
last_token_logits = out[0, -1, :]

# Softmax the logits
probs = torch.softmax(last_token_logits, dim=0)

# Sample the argmax
token_id = torch.argmax(probs)

# Deocode the sample
token_out = model.tokenizer.decode(token_id)
token_out

' ora'

In [65]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((3072,), eps=1e-05)
    (

In [ ]:
# Now we want to visualize the attention pattern.
with model.trace(prompt, output_attentions=True):
    attn_pattern = model.output.attentions[-1].save()
    
print(attn_pattern)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [68]:
attn_pattern.shape

torch.Size([1, 31, 3072])